这一步主要做数据清洗和数据格式化。age和gender给出的数据形式为多值概率，所以选取概率最大的作为用户的基本属性。picnum和pubtime存在一些不符合预期的特征值，需要先做特征清洗。受限于资源限制，所以训练集只保留了两天的数据，而且按照0.01的比例进行了下采样

In [17]:
import pandas as pd
test_info = pd.read_table('../data/test_info.txt',header=None)
print(test_info)

           0           1          2              3  4   5
0          0  2204868548  464495940  1625011774995  2   4
1          1  1623790870  464553715  1625066054894  2   2
2          2  1637010070  464687570  1625053546711  2  10
3          3  1634251474  464661865  1625047014335  2  10
4          4  2352721164  464692079  1625035383911  2  11
...      ...         ...        ...            ... ..  ..
49995  49995  1640585246  464553715  1625036568698  2   1
49996  49996  1987843480  464382231  1625009469794  2   1
49997  49997  2391080328  464617167  1625027566512  5   1
49998  49998   125699342  464172118  1625034692217  2  11
49999  49999  2154437014  464225813  1625008637470  2   3

[50000 rows x 6 columns]


In [18]:
test_info.columns = ['id', 'userid', 'docid', 'timestamp', 'network', 'refresh']
test_info['dt'] = pd.to_datetime(test_info['timestamp'], utc=True,
                               unit='ms').dt.tz_convert('Asia/Shanghai')
test_info['date'] = test_info['dt'].dt.date
test_info['date'] = test_info['date'].astype('str')
print('test data')
print(test_info.head())
test_info.to_pickle('../data/test.pkl')

test data
   id      userid      docid      timestamp  network  refresh  \
0   0  2204868548  464495940  1625011774995        2        4   
1   1  1623790870  464553715  1625066054894        2        2   
2   2  1637010070  464687570  1625053546711        2       10   
3   3  1634251474  464661865  1625047014335        2       10   
4   4  2352721164  464692079  1625035383911        2       11   

                                dt        date  
0 2021-06-30 08:09:34.995000+08:00  2021-06-30  
1 2021-06-30 23:14:14.894000+08:00  2021-06-30  
2 2021-06-30 19:45:46.711000+08:00  2021-06-30  
3 2021-06-30 17:56:54.335000+08:00  2021-06-30  
4 2021-06-30 14:43:03.911000+08:00  2021-06-30  


In [39]:
with open('../data/news_info.txt', 'r') as fd:
    doc_text = fd.read().split('\n')[:-1]

In [40]:
doc_data = list()
for text in doc_text:
    doc_data.append(text.split('\t'))
    
doc_info = pd.DataFrame(doc_data)
doc_info.columns = ['docid', 'title', 'pubtime', 'picnum', 'category1st', 'category2nd', 'keyword']
print(doc_info.shape)

(347466, 7)


In [42]:
# picnum和pubtime存在一些不符合预期的特征值，需要先做特征清洗
# 脏数据
def clean_picnum(x):
    if x in ['上海', '云南', '山东', 'NoneType',''] or x is None: return 0
    else: 
#         print(x)
        return int(float(x))

doc_info['picnum'] = doc_info['picnum'].apply(lambda x: clean_picnum(x))

# 脏数据
def clean_pubtime(x):
    try:   
        x=int(float(x))
        return x
    except:
        print(x)
        return 0
#     if type(x) == str: return 0
#     else: return x

doc_info['pubtime'] = doc_info['pubtime'].apply(lambda x: clean_pubtime(x))


for col in tqdm(['category1st', 'category2nd']):
    lbe = LabelEncoder()
    doc_info[col] = doc_info[col].fillna('NAN')
    doc_info[col] = lbe.fit_transform(doc_info[col])

print('doc info')
print(doc_info.head())
doc_info.to_pickle('../data/doc_info.pkl')

100%|██████████| 2/2 [00:00<00:00, 35.26it/s]


doc info
       docid                         title        pubtime  picnum  \
0  325279629                    合集|外交部每日答问  1624889500000       0   
1  334493096             汽车点评网友“毛蒙”分享的小视频“  1563404900000       0   
2  341014580  抖音爆红车模小姐姐,可爱如神仙,如今大火的她还玩起这个!  1567067600000       3   
3  344297472      一夜能挣30万的网红车模,她的捞金神器竟是这个!  1569240500000       3   
4  347286095  男子购买假号牌被查,后备箱找到真车牌,车主直言不愿太高调  1571271100000       3   

   category1st  category2nd                                            keyword  
0            0            0                                                     
1           23          115                                     https:8.135272  
2           11           56  初恋:11.052198,吊坠:9.525825,吊坠^^耳环:7.982950,妆容:10...  
3           11           56  主播:10.299306,发展:7.355088,可爱:8.705284,清纯:10.108...  
4           23          117  买车:9.357286,交警:8.964902,交通:7.790495,交通^^秩序:10....  


In [10]:
user_info = pd.read_table('../data/user_info.txt',header=None)
print(user_info)

                  0              1        2    3    4  \
0             17340        iPhoneX      IOS   上海   上海   
1            394666       ARS-TL00  Android   吉林   白山   
2            413322       iPhoneXS      IOS   北京   北京   
3            450280       NOH-AN00  Android   湖北   武汉   
4            456646      M2102J2SC  Android   广东   中山   
...             ...            ...      ...  ...  ...   
1029713  2446592934  RedmiNote7Pro  Android   江苏   常州   
1029714  2446592956            NaN      NaN  NaN  NaN   
1029715  2446592962         V2002A  Android   广东   惠州   
1029716  2446592980            NaN      NaN  NaN  NaN   
1029717  2446593010            NaN      NaN  NaN  NaN   

                                                         5  \
0        A_0_24:0.029774,A_25_29:0.126834,A_30_39:0.789...   
1        A_0_24:0.400583,A_25_29:0.261192,A_30_39:0.268...   
2        A_0_24:0.000076,A_25_29:0.017034,A_30_39:0.972...   
3        A_0_24:0.943772,A_25_29:0.043366,A_30_39:0.011...   
4    

In [21]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

user_info.columns = [
    'userid', 'device', 'os', 'province', 'city', 'age', 'gender'
]

def get_cate(x):
    if type(x) == float:
        return x
    li = x.split(',')
    res = list()
    for i in li:
        lbl, prob = i.split(':')
        res.append([lbl, float(prob)])
    res = sorted(res, key=lambda x: x[1])
    return res[-1][0]


user_info['age'] = user_info['age'].apply(lambda x: get_cate(x))
user_info['gender'] = user_info['gender'].apply(lambda x: get_cate(x))

# label encoding 数字化
for col in tqdm(
    ['device', 'os', 'province', 'city', 'age', 'gender']):
    lbe = LabelEncoder()
    user_info[col] = user_info[col].fillna('NAN')
    user_info[col] = lbe.fit_transform(user_info[col])

print('user info')
print(user_info.head())
user_info.to_pickle('../data/user_info.pkl')

100%|██████████| 6/6 [00:02<00:00,  2.96it/s]

user info
   userid  device  os  province  city  age  gender
0   17340    2642   1       185   291    2       2
1  394666     160   0       207   571    0       2
2  413322    2644   1       200   340    2       2
3  450280    1381   0       244   502    0       2
4  456646    1167   0       224   298    2       2


In [1]:
import pandas as pd
train_info = pd.read_table('../data/train_info.txt',header=None)
print(train_info)

                   0          1              2  3  4   5  6    7
0         1000014754  463510256  1624843756147  5  0  16  0    0
1         1000014754  463852707  1624843756147  5  0  13  1   80
2         1000014754  463067100  1624757147178  5  0  13  0    0
3         1000014754  463625484  1624762446340  5  0  12  1  268
4         1000014754  463625484  1624762446340  5  0  12  1   57
...              ...        ...            ... .. ..  .. ..  ...
81671128   999938860  463459430  1624885221612  2  2  20  0    0
81671129   999938860  463930923  1624884995834  2  1  15  1    5
81671130   999938860  463939123  1624885221612  2  2  23  0    0
81671131   999938860  463270375  1624747290961  2  7  67  0    0
81671132   999938860  463247124  1624747290961  2  7  69  0    0

[81671133 rows x 8 columns]


In [2]:
# train_info = train_info.sample(frac=0.1)
# print(train_info)

In [3]:
train_info.columns = ['userid','docid','timestamp','network','refresh','position','click','duration']
print(train_info)

              userid      docid      timestamp  network  refresh  position  \
0         1000014754  463510256  1624843756147        5        0        16   
1         1000014754  463852707  1624843756147        5        0        13   
2         1000014754  463067100  1624757147178        5        0        13   
3         1000014754  463625484  1624762446340        5        0        12   
4         1000014754  463625484  1624762446340        5        0        12   
...              ...        ...            ...      ...      ...       ...   
81671128   999938860  463459430  1624885221612        2        2        20   
81671129   999938860  463930923  1624884995834        2        1        15   
81671130   999938860  463939123  1624885221612        2        2        23   
81671131   999938860  463270375  1624747290961        2        7        67   
81671132   999938860  463247124  1624747290961        2        7        69   

          click  duration  
0             0         0  
1      

In [5]:
# train_info['dt'] = pd.to_datetime(train_info['timestamp'], utc=True, unit='ms').dt.tz_convert('Asia/Shanghai')
# train_info['date'] = train_info['dt'].dt.date
# train_info['date'] = train_info['date'].astype('str')
print('train data')
print(train_info.head())
train_info.to_pickle('../data/train1.0.pkl')

train data
       userid      docid      timestamp  network  refresh  position  click  \
0  1000014754  463510256  1624843756147        5        0        16      0   
1  1000014754  463852707  1624843756147        5        0        13      1   
2  1000014754  463067100  1624757147178        5        0        13      0   
3  1000014754  463625484  1624762446340        5        0        12      1   
4  1000014754  463625484  1624762446340        5        0        12      1   

   duration                               dt        date  
0         0 2021-06-28 09:29:16.147000+08:00  2021-06-28  
1        80 2021-06-28 09:29:16.147000+08:00  2021-06-28  
2         0 2021-06-27 09:25:47.178000+08:00  2021-06-27  
3       268 2021-06-27 10:54:06.340000+08:00  2021-06-27  
4        57 2021-06-27 10:54:06.340000+08:00  2021-06-27  
